In [170]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision.datasets import DatasetFolder
from torchvision.transforms import ToTensor
from random import shuffle
import os
from PIL import Image


import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms.functional as F
import torchvision.transforms as T
from torchvision.io import ImageReadMode, read_image
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import random

In [171]:
forestDirectory = '../../pics/2750/Forest'
industrialDirectory = '../../pics/2750/Industrial'

In [172]:
# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [173]:
# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [174]:
listOfFileNames = []
outputLabels = []

for filename in os.listdir(industrialDirectory):
    imgFile = os.path.join(industrialDirectory, filename)

    if ".jpg" in imgFile:
        listOfFileNames.append(imgFile)
        outputLabels.append([0,1])
        
for filename in os.listdir(forestDirectory):
    imgFile = os.path.join(forestDirectory, filename)

    if ".jpg" in imgFile:
        listOfFileNames.append(imgFile)
        outputLabels.append([1,0])

In [175]:
# Eerst de afbeeldingen weer inladen
industrial = torchvision.io.read_image(f'{industrialDirectory}/Industrial_1.jpg', ImageReadMode.GRAY).float()

i=random.randint(0,2500)
forest = torchvision.io.read_image(f'{forestDirectory}/Forest_{i}.jpg', ImageReadMode.GRAY).float()

forest1 = torchvision.io.read_image(f'{forestDirectory}/Forest_1.jpg', ImageReadMode.GRAY).float()


print(industrial.shape)
print(forest.shape)

torch.Size([1, 64, 64])
torch.Size([1, 64, 64])


In [176]:
listOfFileNames = []
outputLabels = []

for filename in os.listdir(industrialDirectory):
    imgFile = os.path.join(industrialDirectory, filename)

    if ".jpg" in imgFile:
        listOfFileNames.append(imgFile)
        outputLabels.append([0,1])
        
for filename in os.listdir(forestDirectory):
    imgFile = os.path.join(forestDirectory, filename)

    if ".jpg" in imgFile:
        listOfFileNames.append(imgFile)
        outputLabels.append([1,0])

In [177]:
display(len(listOfFileNames))

display(len(outputLabels))

display(type(outputLabels[0]))

5500

5500

list

In [178]:
# Split the dataset into training and validation sets
train_ratio = 0.8  # Ratio of training set size to combined dataset size
train_size = int(train_ratio * len(combined_dataset))
valid_size = len(combined_dataset) - train_size

train_dataset, val_dataset = data.random_split(combined_dataset, [train_size, valid_size])

In [179]:
# Create data loaders

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)


In [180]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        self.drop1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(120, 84)
        self.drop2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(84, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)  

In [181]:
import torch.optim as optim

In [182]:
# Define your model
model = CNN()
model.to(device)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2704, out_features=120, bias=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (drop2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)

In [183]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 256)  # Adjusted input size for linear layer
        self.drop1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256, 84)
        self.drop2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)



# Create an instance of the CNN model
model = CNN()

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Set the number of epochs
num_epochs = 20

In [184]:
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for images, labels, *_ in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x169 and 2704x256)

In [ ]:
# Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_dataloader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)
    
    # Print the loss and epoch
    print(f"Epoch: {epoch+1}/{num_epochs} | Training Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

# Truncate the losses if the training loop was interrupted
if len(train_losses) > num_epochs:
    train_losses = train_losses[:num_epochs]
if len(val_losses) > num_epochs:
    val_losses = val_losses[:num_epochs]

# Plot the losses
fig, ax = plt.subplots(figsize=(8, 6))
ax.set(xlabel='Epoch', ylabel='Loss', title='Loss Function')
ax.plot(range(1, len(train_losses) + 1), train_losses, color='blue', linewidth=2, marker='o', label='Training Loss')
ax.plot(range(1, len(val_losses) + 1), val_losses, color='red', linewidth=2, marker='o', label='Validation Loss')
ax.legend()
ax.grid(True)
plt.show()

